In [1]:
import numpy as np
import random as rm
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
import matplotlib.pyplot as plt
%matplotlib inline
from cvxopt import matrix, solvers
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
import operator
import sys
import pickle
from sklearn.model_selection import KFold
from operator import itemgetter

# Extract from my NN assignment

In [2]:
class Gaussian():
    def __call__(self, a, b, sigma):
        x = np.array(a).reshape(1,-1)
        y = np.array(b).reshape(1,-2)
        return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))
    def name(self):
        return "Gaussian"
    
class MV_Gaussian():
    def __call__(self, a, b, cov):
        x = np.array(a)
        y = np.array(b)
        c = np.array(cov)
        return np.exp(-0.5*np.dot(np.dot((x-y).T, np.linalg.inv(c)), (x-y)))
    
class Linear:
    def __call__(self, a, b, sigma):
        return np.dot(a,b)
    def name(self):
        return "Linear"
    
class Polynomial:
    def __call__(self, a, b, throwaway, p=2):
        self.p = p
        x = np.array(a)
        y = np.array(b)
        y = np.transpose(y)
        return (1 + np.dot(x, y)) ** p
    def name(self):
        return "Quadratic"

In [3]:
csv = np.genfromtxt ('data118448.csv', delimiter=",")
np.random.shuffle(csv)
csv = csv[:200, :]
x = csv[:, :-1]
y = csv[:, -1:]
for i in range(len(x[0])):
    sd = np.std(x[:,i])
    mean = np.mean(x[:,i])
    for j in range(len(x)):
        x[j,i] = (x[j,i]-mean)/sd
train_x = np.copy(x[:-100, :])
train_y = np.copy(y[:-100, :])
top_secret_unseen_x = np.copy(x[-100:, :])
top_secret_unseen_y = np.copy(y[-100:, :])

# add bias to X once pre-processed
ones = np.ones(len(train_x)).reshape(1,-1)
train_x = np.concatenate((train_x, ones.T), axis=1)
ones = np.ones(len(top_secret_unseen_x)).reshape(1,-1)
top_secret_unseen_x = np.concatenate((top_secret_unseen_x, ones.T), axis=1)

## SVR
Support Vector Machines are where it's at. They're cool, so let's try and implement one. But not just any SVR, SVR+ which is mentioned in [this](http://www.sciencedirect.com/science/article/pii/S0893608009001130) paper. Below compares the objective function of each. The plan is to try and be a bit creative and see if we can do something cutting edge... First we're going to build the SVR+. I can't find any implementations online so let's try and build one from just the description. Apologies if the notation is a little confusing, I haven't explicitly said when vectors are column and when they're row (or even when we're talking about matrices!), but this is mostly just to help me work out what I want to do. See the code for a more explicit description.

The idea is then that we're going to use the idea of [LuFE](https://www.ijcai.org/Proceedings/16/Papers/294.pdf). 

### Standard SVR
$$
\begin{aligned}
\min_{w,b,\xi, \xi^*} \quad & \frac{1}{2}(w,w)+C\sum_{i=1}^{\ell}(\xi_i + \xi^*_i) \\
\textrm{subject to} \quad & y_i - (w,x_i) - b \leq \epsilon + \xi_i \\
\textrm{and} \quad & (w,x_i) + b - y_i \leq \epsilon + \xi^*_i \\
\end{aligned}$$

### SVR+
$$
\begin{aligned}
\min_{w, w^*_1, w^*_2,b,b^*_1,b^*_2} \quad & \frac{1}{2}[(w,w)+\gamma[(w^*_1,w^*_1)+(w^*_2,w^*_2)]]+C\sum_{i=1}^{\ell}[(w^*_1,x^*_i)+b^*_1]+C\sum_{i=1}^{\ell}[(w^*_2,x^*_i)+b^*_2] \\
\textrm{subject to} \quad & y_i - (w,x_i) - b \leq \epsilon + (w^*_1,x^*_i)+b^*_1 \\
\textrm{and} \quad & (w,x_i) + b - y_i \leq \epsilon + (w^*_2, x^*_i)+b^*_2 \\
\textrm{and} \quad & [(w^*_1,x^*_i)+b^*_1] \geq 0 \\
\textrm{and} \quad & [(w^*_2,x^*_i)+b^*_2] \geq 0 \\
\end{aligned}
$$

### SVR+ Dual Form

$$
\begin{aligned}
\max_{\alpha, \alpha^*, \beta, \beta^*} \quad & \sum_{i=1}^{\ell} y_i(\alpha^*_i - \alpha_i) - \epsilon \sum_{i=1}^{\ell}(\alpha_i + \alpha_i^*) \\
& - \frac{1}{2}\sum_{i,j=1}^{\ell}(\alpha^*_i - \alpha_i)(\alpha^*_j - \alpha_j) K(x_i,x_j)\\
& - \frac{1}{2\gamma}\sum_{i,j=1}^{\ell}(\alpha_i^* + \beta_i^* -C)(\alpha_j^* + \beta_j^* -C) K^*(x^*_i , x^*_j) \\
& - \frac{1}{2\gamma}\sum_{i,j=1}^{\ell}(\alpha_i + \beta_i -C)(\alpha_j + \beta_j -C) K^*(x^*_i , x^*_j) \\
\textrm{subject to} \quad & \sum_{i=1}^{\ell}\alpha_i^* = \sum_{i=1}^{\ell}\alpha_i \\
\textrm{and} \quad & \sum_{i=1}^{\ell} (\alpha^*_i + \beta_i^* - C) = 0 \\
\textrm{and} \quad & \sum_{i=1}^{\ell} (\alpha_i + \beta_i - C) = 0 \\
\textrm{and} \quad & \alpha \geq 0, \quad \alpha^* \geq 0, \quad \beta \geq 0, \quad \beta^* \geq 0
\end{aligned}
$$

We'll solve this using CVXOPT, which solves problems in the form 

$$
\begin{aligned}
\min_{x} \quad & \frac{1}{2}x^TPx + q^Tx \\
\textrm{subject to} \quad & Gx \leq h \\
& Ax = b
\end{aligned}
$$

So let's put the SVR Dual Form in the correct structure

#### In CVXOPT Form

$$
\begin{aligned}
\min_{\alpha, \alpha^*, \beta, \beta^*} \quad & \epsilon \sum_{i=1}^{\ell}(\alpha_i + \alpha_i^*) - \sum_{i=1}^{\ell} y_i(\alpha^*_i - \alpha_i) \\
& + \frac{1}{2}\sum_{i,j=1}^{\ell}(\alpha^*_i - \alpha_i)(\alpha^*_j - \alpha_j) K(x_i,x_j)\\
& + \frac{1}{2\gamma}\sum_{i,j=1}^{\ell}(\alpha_i^* + \beta_i^* -C)(\alpha_j^* + \beta_j^* -C) K^*(x^*_i , x^*_j) \\
& + \frac{1}{2\gamma}\sum_{i,j=1}^{\ell}(\alpha_i + \beta_i -C)(\alpha_j + \beta_j -C) K^*(x^*_i , x^*_j) \\
\textrm{subject to} \quad & \sum_{i=1}^{\ell}\alpha_i^* - \sum_{i=1}^{\ell}\alpha_i = 0 \\
\textrm{and} \quad & \sum_{i=1}^{\ell} (\alpha^*_i + \beta_i^* - C) = 0 \\
\textrm{and} \quad & \sum_{i=1}^{\ell} (\alpha_i + \beta_i - C) = 0 \\
\textrm{and} \quad & -\alpha \leq 0, \quad -\alpha^* \leq 0, \quad -\beta \leq 0, \quad -\beta^* \leq 0
\end{aligned}
$$

#### Define Variables
By denoting $\delta = \beta - C$ we can describe the variable to find, $x$, as

$$ x = 
\begin{pmatrix}
\alpha \\ \alpha^* \\ \delta \\ \delta^*
\end{pmatrix}
$$

#### Define Objective

$$
P = 
\begin{pmatrix}
(K(x_i,x_j) + \gamma K^*(x^*_i,x^*_j)) & -K(x_i,x_j) & \gamma K^*(x^*_i,x^*_j) & \textbf{0} \\
- K(x_i,x_j) & (K(x_i,x_j) + \gamma K^*(x^*_i,x^*_j)) & \textbf{0} & \gamma K^*(x^*_i,x^*_j) \\
\gamma K^*(x^*_i,x^*_j) & \textbf{0} & \gamma K^*(x^*_i,x^*_j) & \textbf{0} \\
\textbf{0} & \gamma K^*(x^*_i,x^*_j) & \textbf{0} & \gamma K^*(x^*_i,x^*_j)
\end{pmatrix}
$$

$$
q = 
\begin{pmatrix}
(\textbf{Y}+\epsilon ) \\ (-\textbf{Y}+\epsilon ) \\ \textbf{0} \\ \textbf{0}
\end{pmatrix}
$$

$$
\min_{\alpha, \alpha^*, \delta, \delta^*} 
\begin{pmatrix}
\alpha \\ \alpha^* \\ \delta \\ \delta^*
\end{pmatrix}^T
\begin{pmatrix}
(K(x_i,x_j) + \gamma K^*(x^*_i,x^*_j)) & -K(x_i,x_j) & \gamma K^*(x^*_i,x^*_j) & \textbf{0} \\
- K(x_i,x_j) & (K(x_i,x_j) + \gamma K^*(x^*_i,x^*_j)) & \textbf{0} & \gamma K^*(x^*_i,x^*_j) \\
\gamma K^*(x^*_i,x^*_j) & \textbf{0} & \gamma K^*(x^*_i,x^*_j) & \textbf{0} \\
\textbf{0} & \gamma K^*(x^*_i,x^*_j) & \textbf{0} & \gamma K^*(x^*_i,x^*_j)
\end{pmatrix}
\begin{pmatrix}
\alpha \\ \alpha^* \\ \delta \\ \delta^*
\end{pmatrix}
+
\begin{pmatrix}
(\textbf{Y}+\epsilon ) \\ (-\textbf{Y}+\epsilon ) \\ \textbf{0} \\ \textbf{0}
\end{pmatrix}^T
\begin{pmatrix}
\alpha \\ \alpha^* \\ \delta \\ \delta^*
\end{pmatrix}
$$

#### Define Inequalities

$$
G = 
\begin{pmatrix}
\textbf{-I} & \textbf{0} & \textbf{0} & \textbf{0} \\
\textbf{0} & \textbf{-I} & \textbf{0} & \textbf{0} \\
\textbf{0} & \textbf{0} & \textbf{-I} & \textbf{0} \\
\textbf{0} & \textbf{0} & \textbf{0} & \textbf{-I}
\end{pmatrix}
$$

$$
h =
\begin{pmatrix}
\textbf{0} \\
\textbf{0} \\
\textbf{C} \\
\textbf{C}
\end{pmatrix}
$$

$$
\begin{pmatrix}
\textbf{-I} & \textbf{0} & \textbf{0} & \textbf{0} \\
\textbf{0} & \textbf{-I} & \textbf{0} & \textbf{0} \\
\textbf{0} & \textbf{0} & \textbf{-I} & \textbf{0} \\
\textbf{0} & \textbf{0} & \textbf{0} & \textbf{-I}
\end{pmatrix}
\begin{pmatrix}
\alpha \\ \alpha^* \\ \delta \\ \delta^*
\end{pmatrix}
=
\begin{pmatrix}
\textbf{0} \\
\textbf{0} \\
\textbf{C} \\
\textbf{C}
\end{pmatrix}
$$

#### Define Equalities

$$
A = 
\begin{pmatrix}
\textbf{-1} & \textbf{1} & \textbf{0} & \textbf{0} \\
\textbf{1} & \textbf{0} & \textbf{1} & \textbf{0} \\
\textbf{0} & \textbf{1} & \textbf{0} & \textbf{1} 
\end{pmatrix}
$$

$$
b = 
\begin{pmatrix}
0 \\ 0 \\ 0
\end{pmatrix}
$$

$$
\begin{pmatrix}
\textbf{-1} & \textbf{1} & \textbf{0} & \textbf{0} \\
\textbf{1} & \textbf{0} & \textbf{1} & \textbf{0} \\
\textbf{0} & \textbf{1} & \textbf{0} & \textbf{1} 
\end{pmatrix}
\begin{pmatrix}
\alpha \\ \alpha^* \\ \delta \\ \delta^*
\end{pmatrix}
=
\begin{pmatrix}
0 \\ 0 \\ 0
\end{pmatrix}
$$

In [4]:
class SVR:
    def train(self, X, Xstar, Y, C, epsilon, gamma, kern):
        self.training_X = X
        self.Y = Y
        self.C = C
        self.epsilon = epsilon
        self.kern_x = kern
        
        self.sigma = 0
        if kern.name() == "Gaussian":
            sqd = np.zeros((len(X), len(X)))
            for i in range(len(X)):
                for j in range(len(X)):
                    sqd[i, j] = np.linalg.norm(X[i] - X[j])
            self.sigma = np.median(sqd)
        
        x_i_x_j = self.gram(X, X, kern, self.sigma)
        x_star_i_x_star_j = self.gram(Xstar, Xstar, Polynomial(), self.sigma)
        
        p00 = x_i_x_j + gamma*x_star_i_x_star_j
        p01 = -x_i_x_j
        p02 = gamma*x_star_i_x_star_j
        p03 = np.zeros((len(X), len(X)))
        p10 = -x_i_x_j
        p11 = x_i_x_j + gamma*x_star_i_x_star_j
        p12 = np.zeros((len(X), len(X)))
        p13 = gamma*x_star_i_x_star_j
        p20 = gamma*x_star_i_x_star_j
        p21 = np.zeros((len(X), len(X)))
        p22 = gamma*x_star_i_x_star_j
        p23 = np.zeros((len(X), len(X)))
        p30 = np.zeros((len(X), len(X)))
        p31 = gamma*x_star_i_x_star_j
        p32 = np.zeros((len(X), len(X)))
        p33 = gamma*x_star_i_x_star_j
        
        p0 = np.hstack((p00, p01))
        p0 = np.hstack((p0, p02))
        p0 = np.hstack((p0, p03))

        p1 = np.hstack((p10, p11))
        p1 = np.hstack((p1, p12))
        p1 = np.hstack((p1, p13))
        
        p2 = np.hstack((p20, p21))
        p2 = np.hstack((p2, p22))
        p2 = np.hstack((p2, p23))
        
        p3 = np.hstack((p30, p31))
        p3 = np.hstack((p3, p32))
        p3 = np.hstack((p3, p33))
        
        p = np.vstack((p0, p1))
        p = np.vstack((p, p2))
        p = np.vstack((p, p3))
        
        q0 = Y.reshape(1,-1)+epsilon
        q1 = -Y.reshape(1,-1)+epsilon
        q = np.hstack((q0, q1))
        q = np.hstack((q, np.zeros(len(X)).reshape(1,-1)))
        q = np.hstack((q, np.zeros(len(X)).reshape(1,-1)))
        q = q.T
        
        g0 = -np.eye(len(X))
        g0 = np.hstack((g0, np.zeros((len(X), len(X)))))
        g0 = np.hstack((g0, np.zeros((len(X), len(X)))))
        g0 = np.hstack((g0, np.zeros((len(X), len(X)))))
        h0 = np.zeros(len(X)).reshape(-1,1)
        
        g1 = np.zeros((len(X), len(X)))
        g1 = np.hstack((g1, -np.eye(len(X))))
        g1 = np.hstack((g1, np.zeros((len(X), len(X)))))
        g1 = np.hstack((g1, np.zeros((len(X), len(X)))))
        h1 = np.zeros(len(X)).reshape(-1,1)
        
        g2 = np.zeros((len(X), len(X)))
        g2 = np.hstack((g2, np.zeros((len(X), len(X)))))
        g2 = np.hstack((g2, -np.eye(len(X))))
        g2 = np.hstack((g2, np.zeros((len(X), len(X)))))
        h2 = np.repeat(C, len(X)).reshape(-1,1)
        
        g3 = np.zeros((len(X), len(X)))
        g3 = np.hstack((g3, np.zeros((len(X), len(X)))))
        g3 = np.hstack((g3, np.zeros((len(X), len(X)))))
        g3 = np.hstack((g3, -np.eye(len(X))))
        h3 = np.repeat(C, len(X)).reshape(-1,1)
        
        g = np.vstack((g0,g1))
        g = np.vstack((g,g2))
        g = np.vstack((g,g3))
        
        h = np.vstack((h0,h1))
        h = np.vstack((h,h2))
        h = np.vstack((h,h3))
        
        a0 = np.hstack((-np.ones(len(X)), np.ones(len(X))))
        a0 = np.hstack((a0, np.zeros(len(X))))
        a0 = np.hstack((a0, np.zeros(len(X))))
        b0 = 0
        
        a1 = np.hstack((np.zeros(len(X)), np.ones(len(X))))
        a1 = np.hstack((a1, np.zeros(len(X))))
        a1 = np.hstack((a1, np.ones(len(X))))
        b1 = 0
        
        a2 = np.hstack((np.ones(len(X)), np.zeros(len(X))))
        a2 = np.hstack((a2, np.ones(len(X))))
        a2 = np.hstack((a2, np.zeros(len(X))))
        b2 = 0
        
        a = np.vstack((a0,a1))
        a = np.vstack((a,a2))
        
        b = np.vstack((b0,b1))
        b = np.vstack((b,b2))
        
        P = matrix(p, tc='d')
        q = matrix(q, tc='d')
        G = matrix(g, tc='d')
        h = matrix(h, tc='d')
        A = matrix(a, tc='d')
        b = matrix(b, tc='d')
        
        
        solvers.options['show_progress'] = False
        sol = solvers.qp(P, q, G, h, A, b)
        alphas_alphastars_deltas_deltastars = np.array(sol['x'])
        self.alphas = np.asarray(alphas_alphastars_deltas_deltastars[:len(X)])
        self.alphastars = np.asarray(alphas_alphastars_deltas_deltastars[len(X):2*len(X)])
        self.deltas = np.asarray(alphas_alphastars_deltas_deltastars[2*len(X):3*len(X)])
        self.deltastars = np.asarray(alphas_alphastars_deltas_deltastars[3*len(X):])
        
        self.bias = self.get_bias()
        #print(self.get_w())
        
    def get_w(self): # Not used as weight in cany calculations - just used to visualise weights associated with each feature
        running_total = np.zeros_like(self.training_X[0])
        for i in range(len(self.training_X)):
            running_total += (self.alphastars[i] - self.alphas[i]) * self.training_X[i]
        return running_total
            
        
    def get_neg_bias(self):
        running_total = 0
        num_sv = 0
        for i in range(len(self.training_X)):
            if (self.alphas[i] > 1e-8 and self.deltas[i] > -self.C):
                running_total += self.Y[i] - self.predict_wo_bias(self.training_X[i]) - self.epsilon
                num_sv +=1
        return running_total / num_sv if num_sv > 0 else running_total
                
    def get_pos_bias(self):
        running_total = 0
        num_sv = 0
        for i in range(len(self.training_X)):
            if (self.alphastars[i] > 1e-8 and self.deltastars[i] > -self.C):
                running_total += self.Y[i] - self.predict_wo_bias(self.training_X[i]) + self.epsilon
                num_sv +=1
        return running_total / num_sv if num_sv > 0 else running_total
    
    def get_bias(self):
        return (self.get_pos_bias() + self.get_neg_bias())/2
        
    def gram(self, a, b, kern, sig):
        phi = np.zeros((len(a), len(a)))
        for i in range(len(a)):
            for j in range(len(a)):
                phi[i,j] = kern(a[i], b[j], sig)
        return phi
    
    def predict_wo_bias(self, x):
        y = 0
        for i in range(len(self.training_X)):
            y += (self.alphastars[i] - self.alphas[i]) * self.kern_x(self.training_X[i], x, self.sigma)
        return y
        
        
    def predict(self, x):
        y = 0
        for i in range(len(self.training_X)):
            y += (self.alphastars[i] - self.alphas[i]) * self.kern_x(self.training_X[i], x, self.sigma)
        return y + self.bias

In [5]:
def test_performance_svr_plus(train_X, train_Xs, train_Y, test_X, test_Y, C, epsilon, gamma, kern):
    model = SVR()
    model.train(train_X, train_Xs, train_Y, C, epsilon, gamma, kern)
    discrepancy = 0
    y_true = []
    y_pred = []
    for i in range(len(test_X)):
        p = model.predict(test_X[i])
        y_true.append(test_Y[i])
        y_pred.append(p)
        discrepancy += 0.5*((model.predict(test_X[i]) - test_Y[i])**2)
    pred = np.asarray(y_pred).reshape(1,-1)
    save = np.concatenate((test_X, pred.T), axis=1)
    save = np.concatenate((save, test_Y), axis=1)
    np.savetxt("SVR+_output.csv", save, delimiter=",")
    return np.sqrt(discrepancy/len(test_X)), r2_score(y_true, y_pred), C, epsilon, gamma, kern

Again, let's check that we can overfit the model. We'll test the model with the training data. Massively penalize not going through each data point and make epsilon (the window of error) very small.

In [6]:
test_performance_svr_plus(train_x[:,:], train_x[:,:], train_y, train_x[:,:], train_y, 1e+10, 1, 1e+0, Gaussian())

Terminated (singular KKT matrix).


(array([ 0.2642103]),
 0.99999286264650744,
 10000000000.0,
 1,
 1.0,
 <__main__.Gaussian at 0x108449908>)

Cool, that looks as though it works as we'd expect. Don't worry about the singular KKT matrix, we've got some very unusual conditions that we're trying to make work. The second line is the weights assiciated with each feature, we'll come back to them with the LUFe idea. But for now let's go about making improvements.

In [7]:
def cross_validate_svr_plus(X, X_star, Y, Cs, epsilons, gammas, kerns, folds=5):
    kf = KFold(n_splits=folds, shuffle=True)
    kf.get_n_splits(X)
    outcomes = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        X_star_train, X_star_test = X_star[train_index], X_star[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        ikf = KFold(n_splits=folds, shuffle=True)
        ikf.get_n_splits(X_train)
        inner_outcomes = []
        for inner_train_index, inner_test_index in ikf.split(X_train):
            inner_X_train, inner_X_test = X_train[inner_train_index], X_train[inner_test_index]
            inner_X_star_train, inner_X_star_test = X_star_train[inner_train_index], X_star_train[inner_test_index]
            inner_Y_train, inner_Y_test = Y_train[inner_train_index], Y_train[inner_test_index]
            inner_results = [test_performance_svr_plus(inner_X_train, inner_X_star_train, inner_Y_train, inner_X_test, inner_Y_test, C, epsilon, gamma, kern) for C in Cs for epsilon in epsilons for gamma in gammas for kern in kerns]
            inner_outcomes = inner_outcomes + [(ir[0], ir[1], ir[2], ir[3], ir[4], ir[5]) for ir in inner_results]
        means_of_inner = [(np.mean([io[0] for io in inner_outcomes if io[2] == c and io[3] == e and io[4] == g and io[5] == k]), 
                          np.mean([io[1] for io in inner_outcomes if io[2] == c and io[3] == e and io[4] == g and io[5] == k]), c, e, g, k) for c in Cs for e in epsilons for g in gammas for k in kerns]
        best_inner = sorted(means_of_inner, key=itemgetter(0))
        best_inner = best_inner[0]
        outcomes.append(test_performance_svr_plus(X_train, X_star_train, Y_train, X_test, Y_test, best_inner[2], best_inner[3], best_inner[4], best_inner[5]))
    return (np.mean([(outcome[0]) for outcome in outcomes]), np.mean([(outcome[1]) for outcome in outcomes]), best_inner[2], best_inner[3], best_inner[4], best_inner[5].name())


In [8]:
cross_validate_svr_plus(train_x[:50,:], train_x[:50,:], train_y[:50,:], [0.5, 1], [0.5, 1], [0.5, 1], [Gaussian(), Linear()])

(102.12350867733528, -0.075080517886867981, 1, 1, 1, 'Gaussian')